In [1]:
#PROPERTY PRICE PREDICTION PROJECT

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
house=pd.read_csv(r"C:\Users\User\Documents\trains.csv")
house.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,PRICE
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
le=LabelEncoder()

In [6]:
house['POSTED_BYle']=le.fit_transform(house.POSTED_BY)
house['BHK_OR_RKle']=le.fit_transform(house.BHK_OR_RK)
house['ADDRESSle']=le.fit_transform(house.ADDRESS)
house.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,PRICE,POSTED_BYle,BHK_OR_RKle,ADDRESSle
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0,2,0,2844
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0,1,0,6576
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0,2,0,2256
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5,2,0,5095
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5,1,0,3830


In [7]:
housedf=house.drop(['ADDRESS','POSTED_BY','BHK_OR_RK','UNDER_CONSTRUCTION','RERA','READY_TO_MOVE','RESALE'],axis='columns')
housedf.head()

,BHK_NO,SQUARE_FT,LONGITUDE,LATITUDE,PRICE,POSTED_BYle,BHK_OR_RKle,ADDRESSle
0,2,1300.236407,12.969910,77.597960,55.0,2,0,2844
1,2,1275.000000,12.274538,76.644605,51.0,1,0,6576
2,2,933.159722,12.778033,77.632191,43.0,2,0,2256
3,2,929.921143,28.642300,77.344500,62.5,2,0,5095
4,2,999.009247,22.592200,88.484911,60.5,1,0,3830


In [8]:
housedf['SQUARE_FT']=housedf['SQUARE_FT'].astype(int)
housedf['LONGITUDE']=housedf['LONGITUDE'].astype(int)
housedf['LATITUDE']=housedf['LATITUDE'].astype(int)
housedf['PRICE']=housedf['PRICE'].astype(int)

In [9]:
#plt.boxplot(housedf)

In [10]:
housedf['price_per_square']=housedf[('PRICE')]*100000/housedf['SQUARE_FT']
housedf['price_per_square']
housedf[housedf.SQUARE_FT/housedf.BHK_NO<300].head()

housedf1=housedf[~(housedf.SQUARE_FT/housedf.BHK_NO<300)]

In [11]:
housedf2=housedf['price_per_square'].describe()
housedf2

count     29451.000000
mean       7501.433584
std       11258.520870
min         -22.257595
25%        3535.353535
50%        5084.745763
75%        8052.269366
max      900000.000000
Name: price_per_square, dtype: float64

In [12]:
outlier_percentage={}
for feature in ['SQUARE_FT','BHK_NO','PRICE']:
    tempData=housedf1.sort_values(by=feature)[feature]
    Q1,Q3=tempData.quantile([0.25,0.75])
    IQR=Q3-Q1
    lowerrange=Q1-(1.5*IQR)
    upperrange=Q1+(1.5*IQR)
    outlier_percentage[feature]=round((((tempData<(Q1-1.5*IQR))|(tempData>(Q3+1.5*IQR))).sum()/tempData.shape[0])*100,2)
outlier_percentage                               

{'SQUARE_FT': 5.65, 'BHK_NO': 0.92, 'PRICE': 10.62}

In [13]:
outlier_percentage={}
for feature in ['LONGITUDE','LATITUDE','ADDRESSle','POSTED_BYle']:
    tempData=housedf1.sort_values(by=feature)[feature]
    Q1,Q3=tempData.quantile([0.25,0.75])
    IQR=Q3-Q1
    lowerrange=Q1-(1.5*IQR)
    upperrange=Q1+(1.5*IQR)
    outlier_percentage[feature]=round((((tempData<(Q1-1.5*IQR))|(tempData>(Q3+1.5*IQR))).sum()/tempData.shape[0])*100,2)
outlier_percentage                               

{'LONGITUDE': 0.5, 'LATITUDE': 10.17, 'ADDRESSle': 0.0, 'POSTED_BYle': 0.0}

In [14]:
#sns.pairplot(housedf)

In [15]:
#sns.heatmap(housedf.corr(),annot=True)

In [16]:
X=housedf[['POSTED_BYle','BHK_NO','SQUARE_FT']]
y=housedf[['PRICE']]

In [17]:
reg=linear_model.LinearRegression()
reg.fit(X,y)

LinearRegression()

In [18]:
reg.coef_

array([[-1.12077854e+02,  7.75737771e+01,  1.39095943e-04]])

In [19]:
reg.intercept_

array([104.27390482])

In [20]:
reg.score(X,y)

0.18197947949750126

In [21]:
reg.predict([[1300,1,2]])

array([[-145519.36196696]])

In [22]:
#DECISION TREE

In [23]:
from sklearn import tree
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [24]:
model=tree.DecisionTreeClassifier(random_state=42)

In [25]:
trees=model.fit(X,y)
trees

DecisionTreeClassifier(random_state=42)

In [26]:
model.score(X,y)


0.32919085939356896

In [ ]:
plt.figure(figsize=(150,100))
tree.plot_tree(trees,filled=True)

[Text(4532.0472039060405, 5362.540540540541, 'X[2] <= 1430.5\ngini = 0.988\nsamples = 29451\nvalue = [4, 3, 2, 4, 9, 9, 9, 28, 38, 33, 42, 57, 108\n102, 120, 189, 198, 179, 220, 168, 267, 193, 308\n254, 261, 464, 327, 306, 317, 229, 492, 253, 402\n287, 246, 639, 327, 234, 354, 227, 534, 219, 435\n263, 212, 669, 271, 245, 331, 223, 496, 182, 306\n193, 221, 551, 208, 189, 253, 156, 529, 115, 250\n165, 130, 548, 155, 161, 223, 130, 435, 93, 220\n114, 117, 461, 137, 116, 168, 127, 377, 98, 143\n77, 97, 414, 104, 133, 102, 76, 362, 74, 107, 65\n63, 296, 72, 61, 96, 98, 770, 1, 1, 1, 795, 1\n1, 652, 598, 440, 443, 411, 358, 292, 228, 194\n183, 196, 178, 150, 1, 168, 115, 144, 1, 113, 71\n110, 69, 89, 57, 41, 67, 48, 51, 38, 38, 36, 29\n35, 18, 15, 37, 13, 21, 15, 12, 30, 10, 28, 12\n6, 20, 4, 15, 10, 11, 30, 7, 12, 6, 4, 22, 3\n9, 6, 9, 18, 5, 7, 3, 4, 22, 3, 4, 5, 2, 19\n1, 3, 2, 2, 15, 6, 11, 1, 2, 3, 1, 14, 2, 6\n1, 3, 14, 2, 1, 7, 2, 1, 2, 11, 1, 5, 1, 2\n1, 15, 1, 3, 3, 8, 2, 5, 10, 1, 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_log_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scale = StandardScaler()


In [ ]:
lr = LinearRegression()
lr.fit(X,y)
pred=lr.predict(X)
pred

In [ ]:
print("The R2score By Linear Regression is",r2_score(y,pred))


In [ ]:
rf = RandomForestRegressor(max_depth=10,random_state=42)
rf.fit(X,y)

In [ ]:
rf_pred = rf.predict(X)
rf_pred

In [ ]:
test = pd.read_csv(r"../input/project-dataset/testt.csv")
test.head()

In [ ]:
Preds = rf.predict(X)

In [ ]:
submission = pd.DataFrame()
submission["TARGET(PRICE_IN_LACS)"] = Preds
submission.to_csv('house_price.csv', index = False)
submission

In [ ]:
X=housedf.iloc[:, 1:2].values
print(X)
y=housedf.iloc[:, 2].values

In [ ]:
#RandomForestRegression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X,y)

In [ ]:
y_pred=regressor.predict(np.array([6.5]).reshape(1, 1))

In [ ]:
regressor.predict(X)

In [ ]:
regressor.score(X,y)